<h1>
  <b>
    OntoConnect with Graph Neural Network (pytorch - biggraph)
  </b>
</h1>

<ul>
<li>Input:
  <ul>
    <li> source.json (Human) </li>
    <li> target.json (Mouse) </li>
  </ul>
</li>
<li>Output:
  <ul>
    <li> alignment </li>
  </ul>
</ul>

<h2> Model Info: </h2>

<ol>
<li>
PyTorch-BigGraph (PBG) is released by Facebook’s research team
</li>
<li>
Graph embedding methods learn a vector representation of each node in a graph by optimizing the objective that the embeddings for pairs of nodes with edges between them are closer together than pairs of nodes without a shared edge.
</li>
<li>
Graph embedding methods are a form of unsupervised learning, in that they learn representations of nodes using only the graph structure and no task-specific “labels” for nodes.
</li>
<li>
Working Principle
  <ol>
      <li>
        List of edges
        <table>
          <tr>
            <td>
              <table>
                <tr>
                  <td> SOURCE </td>
                  <td> EDGE </td>
                  <td> DESTINATION </td>
                </tr>
                <tr>
                  <td> "http://human.owl#NCI_C41452" </td>
                  <td> "self" </td>
                  <td> "http://human.owl#NCI_C41452" </td>
                </tr>
                <tr>
                  <td> "http://human.owl#NCI_C41452" </td>
                  <td> "parent" </td>
                  <td> "http://human.owl#NCI_C22921" </td>
                </tr>
                <tr>
                  <td> "http://human.owl#NCI_C41452" </td>
                  <td> "child" </td>
                  <td> "http://human.owl#NCI_C13003" </td>
                </tr>
                <tr>
                  <td> "http://human.owl#NCI_C41452" </td>
                  <td> "child" </td>
                  <td> "http://human.owl#NCI_C19526" </td>
                </tr>
                <tr>
                  <td> "http://human.owl#NCI_C41452" </td>
                  <td> "eq" </td>
                  <td> "http://human.owl#NCI_C12928" </td>
                </tr>
                <tr>
                  <td> "http://human.owl#NCI_C41452" </td>
                  <td> "disjoint" </td>
                  <td> "http://human.owl#NCI_C41448" </td>
                </tr>
                <tr>
                  <td> "http://human.owl#NCI_C41452" </td>
                  <td> "restriction" </td>
                  <td> "http://human.owl#NCI_C41623" </td>
                </tr>
              </table>
            </td>
            <td>
                <table>
                  <tr>
                  <td colspan=3>
                  <img src='https://drive.google.com/uc?export=view&id=10Tg2CxvbX-sgemDjXxAiAXa25Ai8pdwS' width='600' height='300'/> 
                  </td>
                </tr>
                </table>
            </td>
          </tr>
        </table>
      </li>
      <li>
        Initial Node Embedding (generated from FastText)
        <table>
          <tr>
            <td> Node </td>
            <td> 100d/200d/300d vectors </td>
          </tr>
          <tr>
            <td> http://human.owl#NCI_C41452 </td>
            <td> [, , , , ..] </td>
          </tr>
          <tr>
            <td> ... </td>
            <td> ... </td>
          </tr>
        </table>
      </li>
      <li>
        The main idea of training the Graph embeddings
        <ol>
            <li>
              The edges provided in dataset are considered as positive edges.
            </li>
            <li>
              It generates negative edges between the nodes which are not connected. These random “false” edges as negative training examples along with the true positive edges.
            </li>
            <li>
              Idea is maximize the score of positive edges and minimize the score of negative edges.
            </li>
        </ol>
      </li>
  </ol>
</li>
</ol>

The <b>score function</b> is as follows \\
$ f(\theta_s, \theta_r, \theta_d) = 
sim(g_{s}(\theta_s, \theta_r), g_{d}(\theta_d, \theta_r))$ \\
sim is cos/dot/l2 \\
$g_{(s/d)}$ is operator ~ none/translation/diagonal/linear/affine/complex-diagonal \\

The <b>loss function</b> is as follows \\
$ \mathcal{L} = \sum_{e \in G} \sum_{e' \in S'_{e}} max( f(e) - f(e') + \lambda, 0) $ \\
$G$ id list of edges \\
$S'_{e}$ set of negative edges for every positive edge \\
$f(e)$ score for a positive edge \\
$f(e')$ score for a negative edge \\
$\lambda$ is regularization

<h2> Step-1 ~ ModifyLbl.ipynb</h2>

[Modify Labels](https://colab.research.google.com/drive/1c4NwIZww27Fg3bcbtlCOVSyPBLl7efpM)

---

<ul>
<li> read the input files
  <ul>
      <li> /ip/source.json </li>
      <li> /ip/target.json </li>
  </ul>
</li>
<li> convert the "lbl" and populate "altLbl" </li>
<li> save the output files
  <ul>
      <li> /modifylbl/source.json </li>
      <li> /modifylbl/target.json </li>
  </ul>
</li>
</ul>


```json
"http://human.owl#NCI_C41452": {
        "lbl": " Subependymal_Cell",
        "altLbl": "cell subependymal",
        "iri": "http://human.owl#NCI_C41452",
        "vector": null,
        "entityTyp": "Class",
        "parentCls": [
            "http://human.owl#NCI_C13003"
        ],
        "childCls": [],
        "eqCls": [],
        "disjointCls": [],
        "restriction": [
            "(<http://human.owl#UNDEFINED_part_of>,http://human.owl#NCI_C41448)",
            "(<http://human.owl#UNDEFINED_part_of>,http://human.owl#NCI_C41623)"
        ]
    }
```



<h2> Step-2 ~ CreateDictionary.ipynb </h2>

[Create Dictionary](https://colab.research.google.com/drive/1zaJ9NI7S_zauaN_aGR6gBC0H0qJcMRKx)

---

<ul>
<li> read the input files </li>
  <ul>
      <li> /modifylbl/source.json </li>
      <li> /modifylbl/source.json </li>
  </ul>
  <li> create dictionary of all words present in the all entities. </li>
  <ul>
      <li> No of Unique Words:- 2124 </li>
  </ul>
<li> save the output files </li>
  <ul>
      <li> /dict/dict.txt </li>
  </ul>
</ul>

<h2> Step-3 ~ DictionaryToVector.ipynb </h2>

[Dictionary To Vector](https://colab.research.google.com/drive/1OG38pTjq2g_O89VLJhogkC8JC5RNMYqe)

---

<ul>
<li> read the dictionary file </li>
  <ul>
      <li> /dict/dict.txt </li>
  </ul>
<li> Get the FastText vector for each dictionary word. </li>
<li> save the output file with vectors </li>
  <ul>
      <li> /dict/dict.json </li>
  </ul>
</ul>

<h2> Step-4 ~ EntityToVector.ipynb </h2>

[Entity To Vector](https://colab.research.google.com/drive/1SwffXceB8PkSu15WUY2ZxOSHvND0djV_)

---

<ul>
<li> read the dictionary file </li>
  <ul>
      <li> /dict/dict.json </li>
      <li> /modifylbl/source.json </li>
      <li> /modifylbl/target.json </li>
  </ul>
<li> copy the vectorsint the source and target file </li>
<li> save the output file with vectors </li>
  <ul>
      <li> /fastentity/source_fast.json </li>
      <li> /fastentity/target_fast.json </li>
  </ul>
</ul>

<h2> Step-5 ~ GenWordSim.ipynb </h2>

[Generate Word Similarity](https://colab.research.google.com/drive/1l_8iOytPoh4k0D0T6XCY-tybXI7WC8mB)

---

<ul>
<li> read the vectors for both the source and target </li>
  <ul>
      <li> /fastentity/source_fast.json </li>
      <li> /fastentity/target_fast.json </li>
  </ul>
<li> create a json file that contains top-k similar human entities for each mouse entity</li>
  <ul>
      <li> /output/word_sim/word_sim_cosine.json </li>
  </ul>
</ul>

***
```json
{
    "http://mouse.owl#MA_0001080": {
        "http://human.owl#NCI_C32243": 0.20560630681970293,
        "http://human.owl#NCI_C33727": 0.20639166686063304,
        "http://human.owl#NCI_C33502": 0.21291418700858478,
        "http://human.owl#NCI_C32903": 0.2141350602468982,
        "http://human.owl#NCI_C12719": 0.2657356704378381
}
```
***

<h2> Step-6 ~ GenerateGraphData.ipynb </h2>

[Generate Graph Data](https://colab.research.google.com/drive/1Rya3lelqcC5MRM6w0K-_WS53_zjz3TTK)

---

<ul>
<li> read the structures for both the source and target </li>
  <ul>
      <li> /fastentity/source_fast.json </li>
      <li> /fastentity/target_fast.json </li>
  </ul>
<li> generate edges for for both source and target</li>
  <ul>
      <li> /gnnentity/source_gnn.json </li>
      <li> /gnnentity/target_gnn.json </li>
  </ul>
</ul>

***
```json
"http://human.owl#NCI_C41452": {
"graphEdges": [
            ["http://human.owl#NCI_C41452", "self", "http://human.owl#NCI_C41452"],
            ["http://human.owl#NCI_C41452", "parent", "http://human.owl#NCI_C13003"],
            ["http://human.owl#NCI_C13003", "parent", "http://human.owl#NCI_C41452"]
            ["http://human.owl#NCI_C41452", "restriction", "http://human.owl#NCI_C41448"],
            ["http://human.owl#NCI_C41448", "restriction", "http://human.owl#NCI_C41452"],
            ["http://human.owl#NCI_C41452", "restriction","http://human.owl#NCI_C41623"],
            ["http://human.owl#NCI_C41623", "restriction","http://human.owl#NCI_C41452"]
        ]
}
```
***

<h2> Step-7 ~ EmbedOntoGraph.ipynb </h2>

[Embed Onto Graph](https://colab.research.google.com/drive/1mx_bnqXk6ZNGlNaFHs0Gh3BXOtjaMl6R)

---

<ul>
<li> 
  read both the source and target one by one
  <ul>
    <li> fastentity/source_fast.json </li>
    <li> fastentity/target_fast.json </li>
  </ul>
</li>
<li> Now for each entity in source/target file 
  <ul>
    <li> 
      extract the enitity name ~ NCI_C41452 
    </li>
    <li> 
      get the updated embedding
        <ul>
          <li>
            first, it creates all the necessary files for the training 
            <ul>
              <li> /gnnentity/entity_graph/src/[entity_nm]/graphs/ </li>
            </ul>
          </li>
          <li>
            populate pre-embeddings (FastText) in the h5py file
          </li>
          <li>
            train each graph (entity) with num_epochs (100)
          </li>
          <li>
             retrieve the new embedding
          </li>
        </ul>
    </li>
    <li> 
      store the new embedding it in dictionary for each entity
    </li>
  </ul>
</li>
<li> store the dictionary in a new file, this will contain the new embedding for each entity both for source and target
  <ul>
    <li> /gnnentity/source_gnn_meta.json </li>
    <li> /gnnentity/target_gnn_meta.json </li>
  </ul>
</ul>
</li>

<h2> Step-8 ~ GenMetaSim.ipynb </h2>

[GenMetaSim](https://colab.research.google.com/drive/1QzgpuX0Sutc6JYxcYoHfggjCJFcK78cz)

---

<ul>
<li> read the vectors for both the source and target </li>
  <ul>
      <li> /gnnentity/source_gnn_meta.json </li>
      <li> /gnnentity/target_gnn_meta.json </li>
  </ul>
<li> create a json file that contains top-k (same as word-sim) similar human entities for each mouse entity</li>
  <ul>
      <li> /output/meta_sim/meta_sim_cosine.json </li>
  </ul>
</ul>

***
```json
{
    "http://mouse.owl#MA_0001080": {
        "http://human.owl#NCI_C32243": 0.20560630681970293,
        "http://human.owl#NCI_C33727": 0.20639166686063304,
        "http://human.owl#NCI_C33502": 0.21291418700858478,
        "http://human.owl#NCI_C32903": 0.2141350602468982,
        "http://human.owl#NCI_C12719": 0.2657356704378381
    }
}
```
***

<h2> Step-9 ~ GenCombSim.ipynb </h2>

[Generate Combine Similarity](https://colab.research.google.com/drive/1gIiCL-Q0AhturkStQURgD0CZgo_ntaMV)

<ul>
<li> read the vectors for both the source and target </li>
  <ul>
      <li> /output/word_sim/word_sim_cosine.json </li>
      <li> /output/meta_sim/meta_sim_cosine.json </li>
  </ul>
<li> create a json file that contains top-k similar human entities for each mouse entity</li>
  <ul>
      <li> /output/output_final.json</li>
  </ul>
</ul>

***
```json
[
   {
        "entity1": "http://mouse.owl#MA_0001087",
        "entity2": "http://human.owl#NCI_C12665",
        "measure": 1.0
   }
]
```
***

<h2> Step-10 ~ OntoEvaluation.ipynb </h2>

[Evaluation](https://colab.research.google.com/drive/1eou9zpCW7dLLMudoMwF5yfPBh6lbZ1_4)

<ul>
<li> read the vectors for both the source and target
  <ul>
      <li> /gold_copy/reference.xml </li>
      <li> /output/output_final.json </li>
  </ul>
</li>
<li> It prints the precision, recall and F-measure</li>
</ul>

***
```text
Precision: 0.935
Recall: 0.710
F measure: 0.807
```
***

<h2> Resources </h2>

<ul>
<li>
<b>Main Paper: https://mlsys.org/Conferences/2019/doc/2019/71.pdf </b>
</li>
<li>
https://torchbiggraph.readthedocs.io/en/stable/data_model.html
</li>
<li>
https://torchbiggraph.readthedocs.io/en/latest/scoring.html#interpreting-the-scores
</li>
<li>
https://torchbiggraph.readthedocs.io/en/latest/faq_troubleshooting.html
</li>
<li>
https://github.com/facebookresearch/PyTorch-BigGraph
</li>
<li> https://github.com/facebookresearch/PyTorch-BigGraph/blob/master/docs/source/configuration_file.rst </li>
<li>http://pages.cs.wisc.edu/~shivaram/cs744-fa20-slides/cs744-pytorch-biggraph-notes.pdf</li>
<li>
https://ai.facebook.com/blog/open-sourcing-pytorch-biggraph-for-faster-embeddings-of-extremely-large-graphs
</li
</ul>

<h2> CrtUtil.ipynb </h2>

[Create Utility](https://colab.research.google.com/drive/1L3IUTTRk4Zx7G5tgz6Xzi-IZR1KSMKpo)

<ul>
<li> create constant (OntoSimConstants.py) </li>
<li> create import (OntoSimImports.py)</li>
<li> create parameter file (ontosim.json)</li>
<li> create folders under data folder</li>
</ul>

<h2> Result Analysis </h2>


<table>
<tr>
  <td>|</td>
  <td>|</td>
  <td colspan=3> 100-dimension |</td>
  <td colspan=3> 200-dimension |</td>
  <td colspan=3> 300-dimension |</td>
</tr>
<tr> 
<tr>
  <td> Number of Prediction |</td>
  <td> Similarity Threshold |</td>
  <td> Precision |</td>
  <td> Recall |</td>
  <td> F-measure |</td>
  <td> Precision |</td>
  <td> Recall |</td>
  <td> F-measure |</td>
  <td> Precision |</td>
  <td> Recall |</td>
  <td> F-measure</td>
</tr>
<tr>
  <td rowspan=10> Top-1 |</td>
  <td> 0.99 |</td>
  <td> 0.974 |</td>
  <td> 0.674 |</td>
  <td> 0.796 |</td>
  <td> 0.979 |</td>
  <td> 0.670 |</td>
  <td> 0.795 |</td>
  <td> 0.979 |</td>
  <td> 0.669 |</td>
  <td> 0.795  </td>
</tr>
<tr>
  <td> 0.98 |</td>
  <td> 0.935 |</td>
  <td> 0.710 |</td>
  <td> <b>0.807</b> |</td>
  <td> 0.967 |</td>
  <td> 0.692 |</td>
  <td> 0.806 |</td>
  <td> 0.973 |</td>
  <td> 0.683 |</td>
  <td> 0.803  </td>
</tr>
<tr>
  <td> 0.970 |</td>
  <td> 0.853 |</td>
  <td> 0.730 |</td>
  <td> 0.787 |</td>
  <td> 0.930 |</td>
  <td> 0.715 |</td>
  <td> <b>0.808</b> |</td>
  <td> 0.953 |</td>
  <td> 0.705 |</td>
  <td> <b>0.810</b>  </td>
</tr>
<tr>
  <td> 0.96 |</td>
  <td> 0.751 |</td>
  <td> 0.752 |</td>
  <td> 0.751 |</td>
  <td> 0.880 |</td>
  <td> 0.733 |</td>
  <td> 0.800 |</td>
  <td> 0.916 |</td>
  <td> 0.721 |</td>
  <td> 0.807  </td>
</tr>
<tr>
  <td> 0.95 |</td>
  <td> 0.678 |</td>
  <td> 0.766 |</td>
  <td> 0.719 |</td>
  <td> 0.808 |</td>
  <td> 0.750 |</td>
  <td> 0.777 |</td>
  <td> 0.868 |</td>
  <td> 0.736 |</td>
  <td> 0.797  </td>
</tr>
<tr>
  <td> 0.94 |</td>
  <td> 0.619 |</td>
  <td> 0.775 |</td>
  <td> 0.688 |</td>
  <td> 0.746 |</td>
  <td> 0.764 |</td>
  <td> 0.755 |</td>
  <td> 0.815 |</td>
  <td> 0.750 |</td>
  <td> 0.781  </td>
</tr>
<tr>
  <td> 0.93 |</td>
  <td> 0.578 |</td>
  <td> 0.786 |</td>
  <td> 0.666 |</td>
  <td> 0.692 |</td>
  <td> 0.773 |</td>
  <td> 0.730 |</td>
  <td> 0.754 |</td>
  <td> 0.764 |</td>
  <td> 0.759  </td>
</tr>
<tr>
  <td> 0.92 |</td>
  <td> 0.553 |</td>
  <td> 0.792 |</td>
  <td> 0.651 |</td>
  <td> 0.648 |</td>
  <td> 0.786 |</td>
  <td> 0.711 |</td>
  <td> 0.713 |</td>
  <td> 0.774 |</td>
  <td> 0.742  </td>
</tr>
<tr>
  <td> 0.91 |</td>
  <td> 0.532 |</td>
  <td> 0.804 |</td>
  <td> 0.641 |</td>
  <td> 0.612 |</td>
  <td> 0.794 |</td>
  <td> 0.690 |</td>
  <td> 0.671 |</td>
  <td> 0.783 |</td>
  <td> 0.723  </td>
</tr>
<tr>
  <td> 0.90 |</td>
  <td> 0.512 |</td>
  <td> 0.810 |</td>
  <td> 0.628 |</td>
  <td> 0.581 |</td>
  <td> 0.801 |</td>
  <td> 0.673 |</td>
  <td> 0.641 |</td>
  <td> 0.789 |</td>
  <td> 0.708  </td>
</tr>
<tr>
  <td colspan=11> </td>
</tr>
<tr>
  <td rowspan=10> Top-3 |</td>
  <td> 0.99 |</td>
  <td> 0.976 |</td>
  <td> 0.676 |</td>
  <td> 0.799 |</td>
  <td> 0.981 |</td>
  <td> 0.671 |</td>
  <td> 0.797 |</td>
  <td> 0.982 |</td>
  <td> 0.671 |</td>
  <td> 0.797  </td>
</tr>
<tr>
  <td> 0.98 |</td>
  <td> 0.940 |</td>
  <td> 0.714 |</td>
  <td> <b>0.811</b> |</td>
  <td> 0.969 |</td>
  <td> 0.693 |</td>
  <td> 0.808 |</td>
  <td> 0.976 |</td>
  <td> 0.685 |</td>
  <td> 0.805  </td>
</tr>
<tr>
  <td> 0.97 |</td>
  <td> 0.862 |</td>
  <td> 0.737 |</td>
  <td> 0.795 |</td>
  <td> 0.934 |</td>
  <td> 0.718 |</td>
  <td> <b>0.812</b> |</td>
  <td> 0.956 |</td>
  <td> 0.706 |</td>
  <td> <b>0.812</b>  </td>
</tr>
<tr>
  <td> 0.96 |</td>
  <td> 0.764 |</td>
  <td> 0.765 |</td>
  <td> 0.766 |</td>
  <td> 0.885 |</td>
  <td> 0.737 |</td>
  <td> 0.805 |</td>
  <td> 0.920 |</td>
  <td> 0.724 |</td>
  <td> 0.810  </td>
</tr>
<tr>
  <td> 0.95 |</td>
  <td> 0.697 |</td>
  <td> 0.787 |</td>
  <td> 0.739 |</td>
  <td> 0.815 |</td>
  <td> 0.757 |</td>
  <td> 0.784 |</td>
  <td> 0.873 |</td>
  <td> 0.740 |</td>
  <td> 0.802  </td>
</tr>
<tr>
  <td> 0.94 |</td>
  <td> 0.637 |</td>
  <td> 0.799 |</td>
  <td> 0.709 |</td>
  <td> 0.758 |</td>
  <td> 0.776 |</td>
  <td> 0.767 |</td>
  <td> 0.821 |</td>
  <td> 0.756 |</td>
  <td> 0.787  </td>
</tr>
<tr>
  <td> 0.93 |</td>
  <td> 0.598 |</td>
  <td> 0.813 |</td>
  <td> 0.689 |</td>
  <td> 0.707 |</td>
  <td> 0.790 |</td>
  <td> 0.746 |</td>
  <td> 0.764 |</td>
  <td> 0.774 |</td>
  <td> 0.769  </td>
</tr>
<tr>
  <td> 0.92 |</td>
  <td> 0.574 |</td>
  <td> 0.822 |</td>
  <td> 0.676 |</td>
  <td> 0.665 |</td>
  <td> 0.807 |</td>
  <td> 0.729 |</td>
  <td> 0.725 |</td>
  <td> 0.787 |</td>
  <td> 0.755  </td>
</tr>
<tr>
  <td> 0.91 |</td>
  <td> 0.554 |</td>
  <td> 0.836 |</td>
  <td> 0.667 |</td>
  <td> 0.628 |</td>
  <td> 0.816 |</td>
  <td> 0.709 |</td>
  <td> 0.687 |</td>
  <td> 0.802 |</td>
  <td> 0.740  </td>
</tr>
<tr>
  <td> 0.90 |</td>
  <td> 0.534 |</td>
  <td> 0.844 |</td>
  <td> 0.654 |</td>
  <td> 0.598 |</td>
  <td> 0.825 |</td>
  <td> 0.694 |</td>
  <td> 0.658 |</td>
  <td> 0.811 |</td>
  <td> 0.726  </td>
</tr>
<tr>
  <td colspan=11> </td>
</tr>
<tr>
  <td rowspan=10> Top-5 |</td>
  <td> 0.99 |</td>
  <td> 0.976 |</td>
  <td> 0.676 |</td>
  <td> 0.799 |</td>
  <td> 0.981 |</td>
  <td> 0.671 |</td>
  <td> 0.797 |</td>
  <td> 0.982 |</td>
  <td> 0.671 |</td>
  <td> 0.797  </td>
</tr>
<tr>
  <td> 0.98 |</td>
  <td> 0.940 |</td>
  <td> 0.714 |</td>
  <td> <b>0.811</b> |</td>
  <td> 0.969 |</td>
  <td> 0.693 |</td>
  <td> 0.808 |</td>
  <td> 0.976 |</td>
  <td> 0.685 |</td>
  <td> 0.805  </td>
</tr>
<tr>
  <td> 0.97 |</td>
  <td> 0.863 |</td>
  <td> 0.738 |</td>
  <td> 0.796 |</td>
  <td> 0.934 |</td>
  <td> 0.718 |</td>
  <td> <b>0.812</b> |</td>
  <td> 0.955 |</td>
  <td> 0.707 |</td>
  <td> <b>0.812</b>  </td>
</tr>
<tr>
  <td> 0.96 |</td>
  <td> 0.766 |</td>
  <td> 0.767 |</td>
  <td> 0.767 |</td>
  <td> 0.885 |</td>
  <td> 0.737 |</td>
  <td> 0.805 |</td>
  <td> 0.920 |</td>
  <td> 0.724 |</td>
  <td> 0.810  </td>
</tr>
<tr>
  <td> 0.95 |</td>
  <td> 0.699 |</td>
  <td> 0.790 |</td>
  <td> 0.742 |</td>
  <td> 0.815 |</td>
  <td> 0.757 |</td>
  <td> 0.784 |</td>
  <td> 0.873 |</td>
  <td> 0.740 |</td>
  <td> 0.802  </td>
</tr>
<tr>
  <td> 0.94 |</td>
  <td> 0.640 |</td>
  <td> 0.802 |</td>
  <td> 0.712 |</td>
  <td> 0.760 |</td>
  <td> 0.779 |</td>
  <td> 0.769 |</td>
  <td> 0.822 |</td>
  <td> 0.757 |</td>
  <td> 0.788  </td>
</tr>
<tr>
  <td> 0.93 |</td>
  <td> 0.603 |</td>
  <td> 0.820 |</td>
  <td> 0.695 |</td>
  <td> 0.709 |</td>
  <td> 0.793 |</td>
  <td> 0.749 |</td>
  <td> 0.766 |</td>
  <td> 0.776 |</td>
  <td> 0.771  </td>
</tr>
<tr>
  <td> 0.92 |</td>
  <td> 0.579 |</td>
  <td> 0.830 |</td>
  <td> 0.682 |</td>
  <td> 0.669 |</td>
  <td> 0.812 |</td>
  <td> 0.734 |</td>
  <td> 0.728 |</td>
  <td> 0.790 |</td>
  <td> 0.758  </td>
</tr>
<tr>
  <td> 0.91 |</td>
  <td> 0.562 |</td>
  <td> 0.848 |</td>
  <td> 0.676 |</td>
  <td> 0.632 |</td>
  <td> 0.821 |</td>
  <td> 0.714 |</td>
  <td> 0.690 |</td>
  <td> 0.805 |</td>
  <td> 0.743  </td>
</tr>
<tr>
  <td> 0.90 |</td>
  <td> 0.542 |</td>
  <td> 0.856 |</td>
  <td> 0.664 |</td>
  <td> 0.604 |</td>
  <td> 0.832 |</td>
  <td> 0.700 |</td>
  <td> 0.662 |</td>
  <td> 0.815 |</td>
  <td> 0.730  </td>
</tr>
</table>
